In [1]:
import cv2
import torch
from datetime import datetime
import json
from pathlib import Path
import shutil
import os
import tkinter as tk
from tkinter import messagebox, filedialog
import tkinter.ttk as ttk  # 导入ttk模块
import tkinter.font as tk_font  # 导入tkinter.font模块
import threading
from ultralytics import YOLO  # 修改导入路径
import logging
import pandas as pd  # 添加pandas导入
import matplotlib.pyplot as plt

# 设置 Matplotlib 使用中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题

# 初始化日志记录
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# 初始化配置
CONFIG = {
    "save_dir": "detection_results",
    "model": "yolov8n.pt",  # 预训练模型
    "confidence_threshold": 0.5,
    "camera_index": 0  # 通常前置摄像头为0
}

class ObjectDetector:
    def __init__(self):
        # 创建保存目录
        self.save_path = Path(CONFIG["save_dir"])
        self.save_path.mkdir(exist_ok=True)
        
        # 加载YOLO模型
        try:
            self.model = YOLO(CONFIG["model"]).to('cpu')  # 强制模型在CPU上运行
            logging.info("模型加载成功")
        except Exception as e:
            logging.error(f"模型加载失败: {e}")
            raise
        
        # 获取类别名称
        self.class_names = self.model.names
        
        # 初始化摄像头
        self.initialize_camera()

    def initialize_camera(self):
        """初始化摄像头"""
        try:
            self.cap = cv2.VideoCapture(CONFIG["camera_index"])
            if not self.cap.isOpened():
                raise Exception("无法打开摄像头")
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
            logging.info("摄像头初始化成功")
        except Exception as e:
            logging.error(f"摄像头初始化失败: {e}")
            raise

    def process_frame(self, frame):
        """执行目标检测并返回结果"""
        try:
            results = self.model(frame)
            # 确保 results 是一个列表并且不为空
            if not results or not results[0].boxes:
                logging.error("检测结果为空")
                return pd.DataFrame()
            
            # 获取原始数据并创建 DataFrame
            boxes = results[0].boxes
            data = boxes.data.cpu().numpy()  # 获取原始数据并转换为numpy数组
            df = pd.DataFrame(data, columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class'])
            if 'confidence' not in df.columns:
                logging.error(f"DataFrame列名错误: {df.columns}")
                return pd.DataFrame()
            
            # 添加类别名称
            df['name'] = df['class'].map(self.class_names)
            
            return df[df['confidence'] > CONFIG["confidence_threshold"]]
        except Exception as e:
            logging.error(f"处理帧时出错: {e}")
            return pd.DataFrame()

    def save_results(self, frame, detections):
        """保存截图和检测结果"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S%f")
        
        # 保存图像
        img_path = self.save_path / f"{timestamp}.png"
        cv2.imwrite(str(img_path), frame)
        
        # 保存元数据
        meta = {
            "timestamp": timestamp,
            "detections": detections.to_dict(orient='records')
        }
        with open(self.save_path / f"{timestamp}.json", 'w') as f:
            json.dump(meta, f)

    def run(self, running_flag):
        """主运行循环"""
        while running_flag.is_set():
            ret, frame = self.cap.read()
            if not ret:
                logging.error("无法获取帧")
                break

            # 执行检测
            detections = self.process_frame(frame)
            valid_detections = detections[detections['confidence'] > CONFIG["confidence_threshold"]]

            # 绘制检测框
            for _, row in valid_detections.iterrows():
                x1, y1, x2, y2 = map(int, row[['xmin', 'ymin', 'xmax', 'ymax']])
                label = f"{row['name']} {row['confidence']:.2f}"
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # 显示画面
            cv2.imshow('Real-time Detection', frame)

            # 键盘控制
            key = cv2.waitKey(1)
            if key == ord('s'):  # 按s保存
                self.save_results(frame, detections)
                logging.info("结果已保存")
            elif key == ord('q'):  # 按q退出
                break

        self.cap.release()
        cv2.destroyAllWindows()

class ObjectDetectorApp:
    def __init__(self, root):
        self.root = root
        self.detector = ObjectDetector()
        self.running_flag = threading.Event()  # 用于控制检测线程的事件标志

        # 创建主界面
        self.create_main_interface()

    def create_main_interface(self):
        self.root.title("YOLOv5 Object Detection")
        self.root.geometry("800x600")

        # 创建自定义字体
        custom_font = tk_font.Font(family="SimHei", size=12)  # 使用 SimHei 字体

        # 创建按钮并应用自定义字体
        self.start_button = tk.Button(self.root, text="开始识别", command=self.start_detection, font=custom_font)
        self.start_button.pack(pady=10)

        self.stop_button = tk.Button(self.root, text="停止识别", command=self.stop_detection, state=tk.DISABLED, font=custom_font)
        self.stop_button.pack(pady=10)

        self.save_button = tk.Button(self.root, text="保存结果", command=self.save_results, font=custom_font)
        self.save_button.pack(pady=10)

        self.history_button = tk.Button(self.root, text="历史记录查询", command=self.show_history, font=custom_font)
        self.history_button.pack(pady=10)

        self.customize_button = tk.Button(self.root, text="自定义识别类别", command=self.customize_categories, font=custom_font)
        self.customize_button.pack(pady=10)

        self.stats_button = tk.Button(self.root, text="数据统计分析", command=self.show_stats, font=custom_font)
        self.stats_button.pack(pady=10)

    def start_detection(self):
        if not self.running_flag.is_set():
            self.running_flag.set()
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.NORMAL)
            self.detector.initialize_camera()  # 重新初始化摄像头
            self.detection_thread = threading.Thread(target=self.detector.run, args=(self.running_flag,))
            self.detection_thread.start()

    def stop_detection(self):
        if self.running_flag.is_set():
            self.running_flag.clear()
            self.start_button.config(state=tk.NORMAL)
            self.stop_button.config(state=tk.DISABLED)
            if hasattr(self, 'detection_thread'):
                self.detection_thread.join()
            self.detector.cap.release()
            cv2.destroyAllWindows()
            logging.info("检测已停止并释放资源")

    def save_results(self):
        if self.detector.cap.isOpened():
            ret, frame = self.detector.cap.read()
            if ret:
                detections = self.detector.process_frame(frame)
                self.detector.save_results(frame, detections)
                messagebox.showinfo("保存成功", "结果已保存")
        else:
            messagebox.showwarning("警告", "摄像头未打开")

    def show_history(self):
        history_window = tk.Toplevel(self.root)
        history_window.title("历史记录查询")
        history_window.geometry("800x600")

        # 加载历史记录
        history_files = list(self.detector.save_path.glob("*.json"))
        history_data = []
        for file in history_files:
            with open(file, 'r') as f:
                data = json.load(f)
                history_data.append(data)

        # 创建表格显示历史记录
        tree = ttk.Treeview(history_window, columns=("Timestamp", "Detections"), show='headings')
        tree.heading("Timestamp", text="时间戳")
        tree.heading("Detections", text="检测结果")
        tree.pack(fill=tk.BOTH, expand=True)

        for data in history_data:
            timestamp = data["timestamp"]
            detections = ", ".join([f"{d['name']} ({d['confidence']:.2f})" for d in data["detections"]])
            tree.insert("", "end", values=(timestamp, detections))

    def customize_categories(self):
        customize_window = tk.Toplevel(self.root)
        customize_window.title("自定义识别类别")
        customize_window.geometry("400x300")

        # 添加自定义类别功能
        tk.Label(customize_window, text="类别名称:").pack(pady=5)
        self.category_entry = tk.Entry(customize_window)
        self.category_entry.pack(pady=5)

        tk.Label(customize_window, text="上传样本数据:").pack(pady=5)
        self.sample_button = tk.Button(customize_window, text="选择文件", command=self.select_sample)
        self.sample_button.pack(pady=5)

        self.train_button = tk.Button(customize_window, text="训练模型", command=self.train_model)
        self.train_button.pack(pady=10)

    def select_sample(self):
        self.sample_path = filedialog.askopenfilename()
        if self.sample_path:
            messagebox.showinfo("文件选择", f"已选择文件: {self.sample_path}")

    def train_model(self):
        if hasattr(self, 'sample_path'):
            messagebox.showinfo("训练模型", "开始训练模型...")
            # 这里可以添加训练模型的代码
        else:
            messagebox.showwarning("警告", "请先选择样本数据文件")

    def show_stats(self):
        stats_window = tk.Toplevel(self.root)
        stats_window.title("数据统计分析")
        stats_window.geometry("800x600")

        # 加载历史记录
        history_files = list(self.detector.save_path.glob("*.json"))
        history_data = []
        for file in history_files:
            with open(file, 'r') as f:
                data = json.load(f)
                history_data.extend(data["detections"])

        # 检查历史数据是否为空
        if not history_data:
            messagebox.showinfo("数据统计分析", "没有历史数据")
            return

        # 统计不同物体类别的出现频率
        df = pd.DataFrame(history_data)
        if 'name' not in df.columns:
            logging.error(f"DataFrame列名错误: {df.columns}")
            messagebox.showinfo("数据统计分析", "数据列名错误")
            return
        category_counts = df['name'].value_counts()

        # 创建柱状图
        fig, ax = plt.subplots()
        category_counts.plot(kind='bar', ax=ax)
        ax.set_title('物体类别出现频率')
        ax.set_xlabel('类别')
        ax.set_ylabel('频率')

        # 显示图表
        canvas = FigureCanvasTkAgg(fig, master=stats_window)
        canvas.draw()
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

if __name__ == "__main__":
    root = tk.Tk()
    try:
        app = ObjectDetectorApp(root)
        root.mainloop()
    except FileNotFoundError as e:
        messagebox.showerror("文件未找到", str(e))
    except Exception as e:
        messagebox.showerror("错误", f"发生错误: {e}")

2025-01-31 18:00:17,057 - INFO - 模型加载成功
2025-01-31 18:00:21,818 - INFO - 摄像头初始化成功
2025-01-31 18:00:31,112 - INFO - 摄像头初始化成功



0: 480x640 1 person, 54.4ms
Speed: 2.0ms preprocess, 54.4ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 46.7ms
Speed: 1.0ms preprocess, 46.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 42.5ms
Speed: 1.0ms preprocess, 42.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 43.3ms
Speed: 1.4ms preprocess, 43.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 43.1ms
Speed: 1.0ms preprocess, 43.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 41.9ms
Speed: 1.0ms preprocess, 41.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 40.2ms
Speed: 1.0ms preprocess, 40.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 43.4ms
Speed: 1.0ms preprocess, 43.4ms inference, 1.0ms postprocess per image at shape (1, 3, 48

2025-01-31 18:00:38,041 - INFO - 检测已停止并释放资源
